# 3. 深度学习计算

# 层和块

之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。
在这里，整个模型只有一个输出。
注意，单个神经网络
（1）接受一些输入；
（2）生成相应的标量输出；
（3）具有一组相关 *参数*（parameters），更新这些参数可以优化某目标函数。

然后，当考虑具有多个输出的网络时，
我们利用矢量化算法来描述整层神经元。
像单个神经元一样，层（1）接受一组输入，
（2）生成相应的输出，
（3）由一组可调整参数描述。
当我们使用softmax回归时，一个单层本身就是模型。
然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。

对于多层感知机而言，整个模型及其组成层都是这种架构。
整个模型接受原始输入（特征），生成输出（预测），
并包含一些参数（所有组成层的参数集合）。
同样，每个单独的层接收输入（由前一层提供），
生成输出（到下一层的输入），并且具有一组可调参数，
这些参数根据从下一层反向传播的信号进行更新。

事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。
例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层，
这些层是由*层组*（groups of layers）的重复模式组成。
这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛的识别和检测任务。
目前ResNet架构仍然是许多视觉任务的首选架构。
在其他的领域，如自然语言处理和语音，
层组以各种重复模式排列的类似架构现在也是普遍存在。

为了实现这些复杂的网络，我们引入了神经网络*块*的概念。
*块*（block）可以描述单个层、由多个层组成的组件或整个模型本身。
使用块进行抽象的一个好处是可以将一些块组合成更大的组件，
这一过程通常是递归的。
通过定义代码来按需生成任意复杂度的块，
我们可以通过简洁的代码实现复杂的神经网络。

![多个层被组合成块，形成更大的模型](http://d2l.ai/_images/blocks.svg)


从编程的角度来看，块由*类*（class）表示。
它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，
并且必须存储任何必需的参数。
注意，有些块不需要任何参数。
最后，为了计算梯度，块必须具有反向传播函数。
在定义我们自己的块时，由于自动微分提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

在构造自定义块之前，(**我们先回顾一下多层感知机**)的代码。
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层，
然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。


In [1]:
## 一个简单的两层感知机

import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1719,  0.0631, -0.2184, -0.1255,  0.2625, -0.0102,  0.2537,  0.1314,
         -0.0533, -0.0662],
        [ 0.0338,  0.2080, -0.0491, -0.2104,  0.2280,  0.1018,  0.1739,  0.1283,
         -0.1370, -0.1016]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化`nn.Sequential`来构建我们的模型，
层的执行顺序是作为参数传递的。
简而言之，(**`nn.Sequential`定义了一种特殊的`Module`**)，
即在PyTorch中表示一个块的类，
它维护了一个由`Module`组成的有序列表。
注意，两个全连接层都是`Linear`类的实例，
`Linear`类本身就是`Module`的子类。
另外，到目前为止，我们一直在通过`net(X)`调用我们的模型来获得模型的输出。
这实际上是`net.__call__(X)`的简写。
这个前向传播函数非常简单：
它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。


## **自定义块**

要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能。
1. 将输入数据作为其前向传播函数的参数。
1. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
1. 存储和访问前向传播计算所需的参数。
1. 根据需要初始化模型参数。

在下面的代码片段中，我们从零开始编写一个块。
它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
注意，下面的`MLP`类继承了表示块的类。
我们的实现只需要提供我们自己的构造函数（Python中的`__init__`函数）和前向传播函数。


In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

我们首先看一下前向传播函数，它以`X`作为输入，
计算带有激活函数的隐藏表示，并输出其未规范化的输出值。
在这个`MLP`实现中，两个层都是实例变量。
要了解这为什么是合理的，可以想象实例化两个多层感知机（`net1`和`net2`），
并根据不同的数据对它们进行训练。
当然，我们希望它们学到两种不同的模型。

接着我们[**实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层**]。
注意一些关键细节：
首先，我们定制的`__init__`函数通过`super().__init__()`
调用父类的`__init__`函数，
省去了重复编写模版代码的痛苦。
然后，我们实例化两个全连接层，
分别为`self.hidden`和`self.out`。
注意，除非我们实现一个新的运算符，
否则我们不必担心反向传播函数或参数初始化，
系统将自动生成这些。

我们来试一下这个函数：


In [3]:
net = MLP()
net(X)

tensor([[-0.2142, -0.1846,  0.2711,  0.0100,  0.0532,  0.2540,  0.3019,  0.0761,
         -0.1798,  0.1380],
        [-0.1012, -0.1309,  0.2606,  0.0119,  0.0270,  0.2167,  0.2997, -0.0449,
         -0.1932,  0.2505]], grad_fn=<AddmmBackward0>)

块的一个主要优点是它的多功能性。
我们可以子类化块以创建层（如全连接层的类）、
整个模型（如上面的`MLP`类）或具有中等复杂度的各种组件。
我们在接下来的章节中充分利用了这种多功能性，
比如在处理卷积神经网络时。

## **顺序块**

现在我们可以更仔细地看看`Sequential`类是如何工作的，
回想一下`Sequential`的设计是为了把其他模块串起来。
为了构建我们自己的简化的`MySequential`，
我们只需要定义两个关键函数：

1. 一种将块逐个追加到列表中的函数；
1. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的`MySequential`类提供了与默认`Sequential`类相同的功能。


In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

`__init__`函数将每个模块逐个添加到有序字典`_modules`中。
读者可能会好奇为什么每个`Module`都有一个`_modules`属性？
以及为什么我们使用它而不是自己定义一个Python列表？
简而言之，`_modules`的主要优点是：
在模块的参数初始化过程中，
系统知道在`_modules`字典中查找需要初始化参数的子块。


当`MySequential`的前向传播函数被调用时，
每个添加的块都按照它们被添加的顺序执行。
现在可以使用我们的`MySequential`类重新实现多层感知机。


请注意，`MySequential`的用法与之前为`Sequential`类编写的代码相同。

## **在前向传播函数中执行代码**

`Sequential`类使模型构造变得简单，
允许我们组合新的架构，而不必定义自己的类。
然而，并不是所有的架构都是简单的顺序架构。
当需要更强的灵活性时，我们需要定义自己的块。
例如，我们可能希望在前向传播函数中执行Python的控制流。
此外，我们可能希望执行任意的数学运算，
而不是简单地依赖预定义的神经网络层。

到目前为止，
我们网络中的所有操作都对网络的激活值及网络的参数起作用。
然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，
我们称之为*常数参数*（constant parameter）。
例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。
因此我们实现了一个`FixedHiddenMLP`类，如下所示：


In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

### Summary of Differences
Input Requirements:
 - torch.mm() strictly requires both inputs to be 2D tensors.
 - torch.matmul() supports 1D, 2D, and higher-dimensional tensors, allowing for more flexible and varied matrix multiplication scenarios.

Functionality:
 - torch.mm() is designed specifically for matrix multiplication of 2D tensors.
 - torch.matmul() is a more general function that can handle various types of tensor multiplications, including dot products, matrix-vector multiplications, matrix multiplications, and batched matrix multiplications.

Practical Usage
 - Use torch.mm() when you are sure that both of your inputs are 2D matrices.
 - Use torch.matmul() for more general use cases, especially when dealing with tensors that are not strictly 2D or when performing batched operations.

#### Understanding these differences will help you choose the appropriate function for your specific tensor operations and ensure that your code handles different types of tensor multiplications correctly.

在这个`FixedHiddenMLP`模型中，我们实现了一个隐藏层，
其权重（`self.rand_weight`）在实例化时被随机初始化，之后为常量。
这个权重不是一个模型参数，因此它永远不会被反向传播更新。
然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情：
它运行了一个while循环，在$L_1$范数大于$1$的条件下，
将输出向量除以$2$，直到它满足条件为止。
最后，模型返回了`X`中所有项的和。
注意，此操作可能不会常用于在任何实际任务中，
**我们只展示如何将任意代码集成到神经网络计算的流程中**。


In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2634, grad_fn=<SumBackward0>)

**我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。**

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1070, grad_fn=<SumBackward0>)

## 效率
读者可能会开始担心操作效率的问题。
毕竟，我们在一个高性能的深度学习库中进行了大量的字典查找、
代码执行和许多其他的Python代码。
Python的问题[全局解释器锁](https://wiki.python.org/moin/GlobalInterpreterLock)
是众所周知的。
在深度学习环境中，我们担心速度极快的GPU可能要等到CPU运行Python代码后才能运行另一个作业。


1. 如果将MySequential中存储块的方式更改为Python列表，会出现什么样的问题？

2. 实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块。

3. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

In [10]:
### Question1

class MySequential2(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for _, module in enumerate(args):
            self.modules.append(module)

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self.modules:
            X = block(X)
        return X

net2 = MySequential2()
net2(X)

tensor([[0.6793, 0.5776, 0.0299, 0.6901, 0.4119, 0.8059, 0.9413, 0.8777, 0.3409,
         0.0738, 0.2525, 0.8563, 0.5847, 0.9011, 0.8983, 0.5051, 0.5203, 0.1404,
         0.5656, 0.6879],
        [0.4903, 0.4111, 0.8778, 0.4911, 0.3628, 0.8324, 0.8178, 0.0507, 0.1604,
         0.5660, 0.4997, 0.1527, 0.5298, 0.4181, 0.0701, 0.2927, 0.4525, 0.8951,
         0.6622, 0.1924]])

In [11]:
net1 = MySequential()
net1(X)

tensor([[0.6793, 0.5776, 0.0299, 0.6901, 0.4119, 0.8059, 0.9413, 0.8777, 0.3409,
         0.0738, 0.2525, 0.8563, 0.5847, 0.9011, 0.8983, 0.5051, 0.5203, 0.1404,
         0.5656, 0.6879],
        [0.4903, 0.4111, 0.8778, 0.4911, 0.3628, 0.8324, 0.8178, 0.0507, 0.1604,
         0.5660, 0.4997, 0.1527, 0.5298, 0.4181, 0.0701, 0.2927, 0.4525, 0.8951,
         0.6622, 0.1924]])

# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2431],
        [0.2396]], grad_fn=<AddmmBackward0>)

## **参数访问**

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [17]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[-0.4685,  0.1523,  0.4276,  0.2170],
        [ 0.2070, -0.2701, -0.3235,  0.4819],
        [-0.1636, -0.4048,  0.0479, -0.0216],
        [-0.3541, -0.3916,  0.2523,  0.2355],
        [ 0.0178, -0.3427, -0.2149, -0.2686],
        [ 0.0372, -0.2371,  0.3755,  0.1828],
        [-0.2516, -0.0220, -0.3187, -0.4787],
        [ 0.3847,  0.0247,  0.3976, -0.3983]])), ('bias', tensor([ 0.1047, -0.3815, -0.2733, -0.3953, -0.4018, -0.3557,  0.4152, -0.3523]))])


In [14]:
print(net[1].state_dict()) ## ReLU层没有参数

OrderedDict()


In [15]:
print(net[2].state_dict()) 
## 参数和输入维度是转置关系，这样矩阵才能相乘。所以这里的矩阵是1*8而不是我们定义的8*1

OrderedDict([('weight', tensor([[ 0.3475, -0.2010,  0.1273, -0.2447,  0.2884,  0.0401,  0.2663,  0.2480]])), ('bias', tensor([0.1481]))])


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### **目标参数**

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [18]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)  ## 我们可以使用.data来访问这个参数的值

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1481], requires_grad=True)
tensor([0.1481])


In [20]:
print(net[2].weight)
print(net[2].weight.data)
print(net[2].weight.data.shape)

Parameter containing:
tensor([[ 0.3475, -0.2010,  0.1273, -0.2447,  0.2884,  0.0401,  0.2663,  0.2480]],
       requires_grad=True)
tensor([[ 0.3475, -0.2010,  0.1273, -0.2447,  0.2884,  0.0401,  0.2663,  0.2480]])
torch.Size([1, 8])


参数是复合的对象，包含值、梯度和额外信息。
这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。


In [21]:
net[2].weight.grad == None

True

### **一次性访问所有参数**

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [22]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [24]:
print(*[(name, param) for name, param in net[0].named_parameters()])
print(*[(name, param) for name, param in net.named_parameters()])

('weight', Parameter containing:
tensor([[-0.4685,  0.1523,  0.4276,  0.2170],
        [ 0.2070, -0.2701, -0.3235,  0.4819],
        [-0.1636, -0.4048,  0.0479, -0.0216],
        [-0.3541, -0.3916,  0.2523,  0.2355],
        [ 0.0178, -0.3427, -0.2149, -0.2686],
        [ 0.0372, -0.2371,  0.3755,  0.1828],
        [-0.2516, -0.0220, -0.3187, -0.4787],
        [ 0.3847,  0.0247,  0.3976, -0.3983]], requires_grad=True)) ('bias', Parameter containing:
tensor([ 0.1047, -0.3815, -0.2733, -0.3953, -0.4018, -0.3557,  0.4152, -0.3523],
       requires_grad=True))
('0.weight', Parameter containing:
tensor([[-0.4685,  0.1523,  0.4276,  0.2170],
        [ 0.2070, -0.2701, -0.3235,  0.4819],
        [-0.1636, -0.4048,  0.0479, -0.0216],
        [-0.3541, -0.3916,  0.2523,  0.2355],
        [ 0.0178, -0.3427, -0.2149, -0.2686],
        [ 0.0372, -0.2371,  0.3755,  0.1828],
        [-0.2516, -0.0220, -0.3187, -0.4787],
        [ 0.3847,  0.0247,  0.3976, -0.3983]], requires_grad=True)) ('0.bias', P

这为我们提供了另一种访问网络参数的方式，如下所示。

In [25]:
net.state_dict()['0.bias'].data

tensor([ 0.1047, -0.3815, -0.2733, -0.3953, -0.4018, -0.3557,  0.4152, -0.3523])

### **从嵌套块收集参数**

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [26]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1914],
        [0.1914]], grad_fn=<AddmmBackward0>)

In [27]:
print(rgnet.state_dict())

OrderedDict([('0.block 0.0.weight', tensor([[ 0.4534, -0.3344,  0.0375, -0.2485],
        [ 0.2532,  0.0048,  0.2724, -0.1310],
        [ 0.0075, -0.0177,  0.2449, -0.1703],
        [-0.4178, -0.1385,  0.4274, -0.4289],
        [-0.1031, -0.4183,  0.3720,  0.2747],
        [ 0.1418,  0.4607, -0.2048,  0.4437],
        [-0.2948, -0.2325,  0.1778,  0.1342],
        [-0.1966, -0.3470,  0.4549,  0.0425]])), ('0.block 0.0.bias', tensor([-0.1345,  0.3144, -0.2128,  0.3485,  0.0447,  0.2750, -0.1725, -0.4577])), ('0.block 0.2.weight', tensor([[-0.2414,  0.1234, -0.3389, -0.0876, -0.0691,  0.3348, -0.3191, -0.2875],
        [ 0.1691, -0.0538,  0.0482, -0.3294, -0.0346, -0.0166, -0.1113,  0.3231],
        [-0.1308,  0.0048,  0.0403,  0.1629,  0.2921,  0.2143, -0.2688, -0.0754],
        [ 0.0236, -0.1183, -0.0849, -0.2337, -0.1574, -0.3059, -0.0702,  0.3110]])), ('0.block 0.2.bias', tensor([ 0.0278, -0.0063, -0.2406, -0.2715])), ('0.block 1.0.weight', tensor([[-0.3716,  0.0426,  0.4760,  0.1416]

In [28]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### <font color="green"> Exercise: </font>
#### 1. 取出block2的第二个线性层的权重值
#### 2. 取出不属于任何一个block层的线性层的权重值

In [43]:
### 取出block2的第二个线性层的权重值
rgnet.state_dict()["0.block 2.2.weight"].data

tensor([[-0.1610, -0.1948, -0.1043, -0.1215, -0.0282,  0.2583, -0.0252,  0.0601],
        [-0.0071, -0.0540,  0.0233, -0.2840,  0.2067, -0.1861, -0.2885, -0.1971],
        [-0.1084,  0.2378, -0.2678,  0.0487,  0.2908,  0.2647, -0.1901, -0.3354],
        [-0.3097,  0.3002, -0.2387, -0.2053,  0.0755,  0.1713, -0.2548, -0.1546]])

In [44]:
rgnet[0][1][2].weight.data

tensor([[ 0.1799,  0.2817,  0.1826, -0.2483,  0.2277, -0.2317,  0.0343,  0.3402],
        [-0.0447,  0.3260,  0.0039,  0.2002,  0.1368, -0.0641,  0.1508, -0.2149],
        [ 0.3505,  0.1646, -0.0885, -0.0476, -0.2303,  0.0993,  0.2226, -0.1760],
        [-0.3231,  0.1635,  0.0019,  0.1953,  0.2086,  0.2356, -0.3177,  0.1484]])

In [29]:
### 取出不属于任何一个block层的线性层的权重值
rgnet.state_dict()["1.weight"].data

tensor([[ 0.1826,  0.2734, -0.1363, -0.0540]])

因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。(**一个高级操作**)

In [31]:
rgnet[0][1][0].bias.data

tensor([ 0.0311,  0.0300, -0.1807,  0.4228, -0.1311, -0.3480, -0.2724,  0.4388])

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。

默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，
这个范围是根据输入和输出维度计算出的。
PyTorch的`nn.init`模块提供了多种预置初始化方法。

### **内置初始化**

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [32]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  ### _表示inplace=True的操作
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0109,  0.0075, -0.0115, -0.0006]), tensor(0.))

In [33]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

### 但是，我们一般不这样去做，不把参数的初始值定为一样的值。这样，同层所有的神经元 参数 梯度全一样了

(tensor([1., 1., 1., 1.]), tensor(0.))

In [35]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
# print(net[0].weight.data[0])
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.6972, -0.6312, -0.2773, -0.3195],
        [ 0.3861,  0.2339, -0.4569,  0.5866],
        [-0.2406,  0.6648, -0.6771,  0.3910],
        [-0.6705, -0.1885, -0.1816, -0.1126],
        [ 0.0741,  0.5355, -0.5666, -0.6383],
        [-0.0708, -0.1077,  0.3235, -0.4137],
        [-0.0693, -0.3128, -0.2090,  0.1416],
        [-0.1590,  0.6419, -0.4518,  0.0842]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### **自定义初始化**

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


In [41]:
# 同样，我们实现了一个my_init函数来应用到net。
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        ### 这里是只保留权重大于等于5的权重值，相当于乘了一个筛选的矩阵

net.apply(my_init)
print(net[0].weight[:2])
net[0].weight.data

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[-8.4792, -7.6218,  6.4867, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000]], grad_fn=<SliceBackward0>)


tensor([[-8.4792, -7.6218,  6.4867, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000],
        [-5.8266,  6.6505, -5.1802, -5.2352],
        [-0.0000, -9.4561, -8.8997, -6.3505],
        [ 0.0000, -7.0935,  8.7218,  5.8544],
        [ 0.0000,  9.1252, -6.1607, -8.6265],
        [ 9.2162,  0.0000,  5.7082, -6.9620],
        [ 0.0000, -0.0000,  8.2085, -0.0000]])

In [37]:
### 注意，我们始终可以直接设置参数。

net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  9.9626,  6.1704])

## **参数绑定**

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [48]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net[2].weight.data == net[4].weight.data)
net[2].weight.data[0] = 100
print(net[2].weight.data)
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])
tensor([[ 1.0000e+02,  1.0000e+02,  1.0000e+02,  1.0000e+02,  1.0000e+02,
          1.0000e+02,  1.0000e+02,  1.0000e+02],
        [ 2.4502e-01, -1.6212e-01,  1.5172e-01,  9.9146e-02, -6.6791e-02,
         -1.4691e-01, -3.2316e-01, -3.4960e-02],
        [ 7.6115e-02,  1.3919e-01,  3.1959e-01, -8.4741e-02, -2.6270e-01,
          2.3121e-01, -2.3470e-03,  3.3919e-01],
        [ 9.5434e-02,  2.9177e-01,  3.1700e-03, -9.1332e-02, -1.8561e-01,
         -1.8819e-01, -3.0975e-01, 

这个例子表明第三个和第五个神经网络层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。
因此，如果我们改变其中一个参数，另一个参数也会改变。
这里有一个问题：当参数绑定时，梯度会发生什么情况？
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层
（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用`sec_model_construction`中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [49]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [50]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))  # X.mean() = 3.0

tensor([-2., -1.,  0.,  1.,  2.])

## 现在，我们可以将层作为组件合并到更复杂的模型中。

In [51]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [52]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [54]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.4838, -0.0714, -0.2881],
        [-0.5628, -1.5581,  0.1893],
        [-0.1200, -1.4623, -0.4105],
        [ 1.4874, -0.5209,  0.6994],
        [ 0.7198, -2.4847, -0.0266]], requires_grad=True)

In [55]:
linear(torch.rand(2, 5))

tensor([[1.5951, 0.0000, 0.0000],
        [0.7091, 0.0000, 0.0000]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [56]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[5.8393],
        [2.0874]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。
